In [ ]:
from sklearn.svm import SVC
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import ml_utils
from sklearn.preprocessing import StandardScaler

## Support Vector Machines

Support vector machines are really common and versitile - they can do classification and regression.

SVM are easiest to understand through looking at some example visualizations. 

In [ ]:
# Load Iris Dataset
df = ml_utils.sklearn_to_df(datasets.load_iris())
df.head()

We will also simplify the data a little, so we can picture the support vectors. We want only 2 features to plot on a scatter plot. 

In [ ]:
d = df.drop(columns={"sepal length (cm)", "sepal width (cm)"})
d = d[d["target"] <= 1]
x_lab = "petal length (cm)"
y_lab = "petal width (cm)"

sns.scatterplot(data=d, y=y_lab, x=x_lab, hue="target")

### Separate These

After we've plotted the data we can intuitively divide the groups - we could sketch a line right through the middle that divides the two groups. These groups are linearly separable, they can be separated by a line. In fact they could be separated by several different lines. 

![SVM Separation](images/svm_bad.png "SVM" )

We can show a separation that the SVM determines by fitting an SVM model to the data, and doing a bunch of plotting work to make it pretty. SVM looks for the separation line (called a hyperplane) that maximizes that margin between the two classes. 

In [ ]:
#SVM Stuff
#Note: this stuff is just to draw the lines, you don't always need all this. 
#Note 2: Plotting this stuff is a pain! There's a library that we'll look at in a bit that makes it easier. 
def plot_svc_decision_boundary(svm_clf, xmin, xmax, color="black"):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w0*x0 + w1*x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    plt.plot(x0, decision_boundary, "-", linewidth=2, color=color)
    plt.plot(x0, gutter_up, "--", linewidth=2, color=color)
    plt.plot(x0, gutter_down, "--", linewidth=2, color=color)

Generate fitted Support Vector Classifier and plot the separation. 

In [ ]:
y = d["target"]
X = d.drop(columns={"target"})
svm_clf = SVC(kernel="linear", C=float("inf"))
svm_clf.fit(X, y)

sns.scatterplot(data=d, y=y_lab, x=x_lab, hue="target")
plot_svc_decision_boundary(svm_clf, 0, 5.5)

### SVM Separation

The above result shows how SVM works in one picture - it draws a line to separate the two groups, and chooses the place for that line that maximizes the width of the dotted lines - the decision boundaries. 

In this example the two classes are very separate, so the SVM can create a nice wide boundary splitting A and B. Note that if we had a bunch more data that was outside of the boundary, it wouldn't change the results - only things that impact the decision boundary "matter" in making the model. 

What we have is a hard margin classification - there is a strict division between classes. 

### The Hyperplane

The hyperplane is the term for that line generated between the two datasets. What is it? It is a linear function in dimensions n-1 that splits the groups. For 2D examples, it is a line. For 3 dimensions, it is a plane. For greater dimensions, it gets hard to picture.

![SVM Hyperplane](images/svm_plane.png "SVM Hyperplane" )

The SVM algorithm tries to maximize this boundary size - this should intuitively make sense. The wider the gulf between the two classes, the more confident we can be in separating an item into one group or another. The items that are close to the boundaries are the "borderline" items that are close and could go either way - we want to minimize that. 

#### Support Vectors

The points that define the boundaries, that are intersected by the dashed lines, are called the support vectors. 

#### SVM Soft Margins

Many to most things we want to classify don't have such a severe and simple separation of classes. With hard margins, even one outlier that was "on the other side" could mess things up. To deal with this we can make our model softer, or allow it to deal with some amount of overlap in our classes. This is (obviously) soft margin classification. 

This is done by setting a hyperparameter C in the model: 
<ul>
<li> High C values minimize overlap, but result in smaller boundaries. 
<li> Low C values allow for more overlap, but have wider boundaries. 
</ul>

In general higher C will make models more fitted, lower will make them more general. 

In [ ]:
d2 = df.drop(columns={"sepal length (cm)", "sepal width (cm)"})
d2 = d2[d2["target"] != 2]
y2 = d2["target"]
X2 = d2.drop(columns={"target"})

svm_wide = SVC(kernel="linear", C=float(5))
svm_wide.fit(X2, y2)
svm_nar = SVC(kernel="linear", C=float(.1))
svm_nar.fit(X2, y2)

sns.scatterplot(data=d2, y=y_lab, x=x_lab, hue="target").set(title="Low C - Wider Margins. Less Fitted")
plot_svc_decision_boundary(svm_nar, 0, 6)



In [ ]:
sns.scatterplot(data=d2, y=y_lab, x=x_lab, hue="target").set(title="High C - Smaller Margins. More Fitted")
plot_svc_decision_boundary(svm_wide, 0, 6, color="red")

### Non Linear Separation

The SVM we just looked at looks pretty good, as long as things are linearly separable. Unfortunately not all data is like that. 

Similarly to using a basis function to "curve" a linear regression to non-linear data, we can use the kernal argument to apply an SVM to a non linear pattern. This is sometimes called the "kernal trick" and is the key to dealing with real data, which often doesn't have nice wide spearations like we did with the sample. 

For example, we can create some dummy data that is not very linearly separable. 

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline


X_moon, y_moon = make_moons(n_samples=100, noise=0.15, random_state=42)
d_moon = pd.DataFrame(X_moon[:,0], columns={"X1"})
d_moon["X2"] = X_moon[:,1]
d_moon["y"] = y_moon
sns.scatterplot(data=d_moon, x="X1", y="X2", hue="y")


#### Separation

These two datasets are nonn linearly separable. If we were to draw a curved line between them it would look kind of like a cubic polynomial. We can use a kernel in our model to incorporate that!

### Kernels

Kernels function like basis functions in linear regression. We can specify this as a hyperparameter when creating the SVM object. The choices are:
<ul>
<li> linear
<li> rbf - radial (circular)
<li> poly - polynomial
<li> sigmoid - like our sigmoid curve in linear regression. 
<li> custom - you can create a custom function to do the transformation you'd like
</ul>

Each of these will generate a different split, we want to choose one that matches our data. With these 2D examples, that isn't too hard; with real data we are probably in a guess and test with a grid search scenario. 

![SVM Kernels](images/svm_kern.png "SVM Kernels" )

Note that the linear one is fast, especially on large datasets, the others are a fair bit slower. 

In [ ]:
poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1))
    ])
poly_kernel_svm_clf.fit(X_moon, y_moon)

### Plotting with Hyperplanes

As we've seen above, plotting these charts can be a pain, there is a library that makes it easier but it does need to be installed. The library is called mlxtend. This one isn't in the conda install stuff, you need to use pip to install it:

> pip install mlxtend

If you don't have pip, that might actually be helpful with installing things in general. It can be installed via:

> conda install pip

If you don't have this installed, it isn't really the end of the world. This visualization stuff is more for the demonstrations of how SVMs work, when using real data it is pretty hard to visualize it like this when there are >3 dimensions.

In [ ]:
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X=X_moon, 
                      y=y_moon,
                      clf=poly_kernel_svm_clf, 
                      legend=2)

#### Radial Kernel

What if we make some data that is circular? 

In [ ]:
from sklearn.datasets import make_circles
X_cir, y_cir = make_circles(n_samples=200, noise=0.15, factor=.3)
d_cir = pd.DataFrame(X_cir[:,0], columns={"X1"})
d_cir["X2"] = X_cir[:,1]
d_cir["y"] = y_cir
sns.scatterplot(data=d_cir, x="X1", y="X2", hue="y")

This is obviously pretty difficult to imposible to separate linearly, but if we give our SVM classifier a radial kernel, it can split these two classes, probably almost perfectly (we have randomness, so each dataset is slightly different).

In [ ]:
radial_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf"))
    ])
radial_kernel_svm_clf.fit(X_cir, y_cir)

plot_decision_regions(X=X_cir, 
                      y=y_cir,
                      clf=radial_kernel_svm_clf, 
                      legend=2)

## Exercise - Weird Shape

The code below will give us a random classification to classify. Try using a grid search with different kernels to see which one fits. Every run will give a different shape of data, so the right answer is random. 

In [ ]:
#Make some random data
from sklearn.datasets import make_classification
X_cla, y_cla = make_classification(n_samples=10000, n_features=2, n_redundant=0, class_sep=.9)
d_cla = pd.DataFrame(X_cla[:,0], columns={"X1"})
d_cla["X2"] = X_cla[:,1]
d_cla["y"] = y_cla
sns.scatterplot(data=d_cla, x="X1", y="X2", hue="y")

In [ ]:
#Use grid search to find svm model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_cla, y_cla)


### ROC Curves

In addition to our friend the confusion matrix, ROC curves are another useful way to visualize classifier performance. ROC stands for Receiver Operating Characteristic and it plots the true positive rate (recall) against the false positive rate (1-specificity). The way we read the curve is pretty simple, the faster the line for our model pushes towards the top left corner, the better it is. What that is measuring is effectively that the model is able to "accumulate" true positives "before" false positives - or the ratio of TP to FP is high, which is what we want. 

The dotted diagonal line represents random chance, and the line for our model represents the improvement over random chance. The random chance line shows that if we were just guessing, TP and FP would be equally likely, so we'd climb to getting 100% of the TP getting an equal amount of FP. The farther up our line is, the more TP we get "for each" FP that the model predicts. 

There isn't a all-in-one chart in seaborn that can print a visualization of a ROC curve in one step, which is annoying. This is the type of problem that is both common and relatively easy, so the internet offers us a million different implementations that we can borrow from if we don't want to make a linechart by hand. 

In [ ]:
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train, train_preds)
plot_roc_curve(fpr,tpr)

### ROC Score

We can also get the ROC score, which measures the area under the curve (AUC). The score goes between 0 and 1, the higher the better. If it was 1, that would indicate that we get all the true positives, with no false positives, which is pretty good!. 

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, train_preds)

## SVM and Multinomial Classifications

Like linear regression, SVMs are a 2 class thing by default. In an SVM classifier we can choose between One vs One, or One vs Rest, with OVR being the default. 

We can try it for the flower example, with all the data. This has 3 output classes and 4 features. We can't really visualize it in 4D very easily but we can make the classification. If we take a peek at the scatterplots and think about the OVR decisions, I'd suspect going in that we'd be able to do this pretty accurately. If we visualize one set vs the other two, each scatter plot looks like we can come pretty close to separating those groups linearly - they are reasonably distinct from each other. Let's see the results...

In [ ]:
sns.pairplot(data=df, hue="target")

In [ ]:

y2 = np.array(df["target"]).reshape(-1,1)
X2 = np.array(df.drop(columns={"target"}))

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2)

flower_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC())
    ])

params = {'svm_clf__kernel':["linear", "poly", "rbf", "sigmoid"],
        "svm_clf__C":[0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 4, 6, 10]}
 
clf2 = GridSearchCV(flower_clf, param_grid=params, cv=5, n_jobs=-1) 
clf2.fit(X_train2, y_train2.ravel())
best2 = clf2.best_estimator_
train_preds2 = best2.predict(X_train2)
test_preds2 = best2.predict(X_test2)
print("Train Score:", best2.score(X_train2, y_train2))
print("Test Score:", best2.score(X_test2, y_test2))
print(best2)
